<a href="https://colab.research.google.com/github/Sottix99/ADM-HW5/blob/main/HW5_ale.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Data

## Data Preprocessing

In [1]:
import pandas as pd
import networkx as nx
import string 
import matplotlib.pyplot as plt
import scipy
from collections import Counter

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
edges = pd.read_csv('/content/drive/MyDrive/hw 5 ADM/edges.csv')

In [4]:
hero_network = pd.read_csv('/content/drive/MyDrive/hw 5 ADM/hero-network.csv')

In [5]:
nodes = pd.read_csv('/content/drive/MyDrive/hw 5 ADM/nodes.csv')

Some of the heroes' names in 'hero-network.csv' are not found in 'edges.csv'. This inconsistency exists for the following reasons:

Some heroes' names in 'hero-netowrk.csv' have extra spaces at the end of their names compared to their names in 'edges.csv'.
Some heroes' names in 'hero-netowrk.csv' have an extra '/' at the end of their names compared to their names in 'edges.csv'.
The hero name 'SPIDER-MAN/PETER PARKER' in 'edges.csv' has been changed to 'SPIDER-MAN/PETER PAR' in 'hero-network.csv' due to a string length limit in 'hero-network.csv'.

In [6]:
#Removing extra spaces at the end of the hero name
hero_network['hero1'] = hero_network['hero1'].apply(lambda x: x.rstrip())
hero_network['hero2'] = hero_network['hero2'].apply(lambda x: x.rstrip())

In [7]:
#Replace 'SPIDER-MAN/PETER PAR' with 'SPIDER-MAN/PETER PARKER'
hero_network.replace({'SPIDER-MAN/PETER PAR': 'SPIDER-MAN/PETER PARKER'}, inplace = True)

In [8]:
#Removing extra '/'
hero_network['hero1'] = hero_network['hero1'].apply(lambda x: x.rstrip('/'))
hero_network['hero2'] = hero_network['hero2'].apply(lambda x: x.rstrip('/'))
edges['hero'] = edges['hero'].apply(lambda x: x.strip('/'))

In [9]:
#After this we need to remove again the empty spaces at the end
hero_network['hero1'] = hero_network['hero1'].apply(lambda x: x.rstrip())
hero_network['hero2'] = hero_network['hero2'].apply(lambda x: x.rstrip())
edges['hero'] = edges['hero'].apply(lambda x: x.rstrip())

In [10]:
set(hero_network['hero1']).union(set(hero_network['hero2'])) == set(edges['hero'])

False

In [11]:
(set(hero_network['hero1']).union(set(hero_network['hero2']))).difference(set(edges['hero']))

set()

In [12]:
len(set(edges['hero']).difference(set(hero_network['hero1']).union(set(hero_network['hero2']))))

18

### Removing duplicates

In [13]:
hero_network.drop((hero_network[hero_network['hero1'] == hero_network['hero2']]).index, inplace = True)

### Save datasets

In [14]:
hero_network.to_csv('/content/drive/MyDrive/hw 5 ADM/hero_net_clean.csv', index=False)

In [15]:
edges.to_csv('/content/drive/MyDrive/hw 5 ADM/edges_clean.csv', index=False)

In [16]:
nodes.to_csv('/content/drive/MyDrive/hw 5 ADM/nodes_clean.csv', index=False)

## Graphs setup

#### First

In [17]:
hero_network

,hero1,hero2
0,"LITTLE, ABNER",PRINCESS ZANDA
1,"LITTLE, ABNER",BLACK PANTHER/T'CHAL
2,BLACK PANTHER/T'CHAL,PRINCESS ZANDA
3,"LITTLE, ABNER",PRINCESS ZANDA
4,"LITTLE, ABNER",BLACK PANTHER/T'CHAL
...,...,...
574462,COLOSSUS II/PETER RA,CALLISTO
574463,CALLISTO,ROGUE
574464,CALLISTO,CALIBAN
574465,CALIBAN,ROGUE


In [18]:
hero_net_g=hero_network.groupby(hero_network.columns.tolist(),as_index=False).size()
hero_net_g

,hero1,hero2,size
0,24-HOUR MAN/EMMANUEL,"FROST, CARMILLA",1
1,24-HOUR MAN/EMMANUEL,KILLRAVEN/JONATHAN R,1
2,24-HOUR MAN/EMMANUEL,M'SHULLA,1
3,3-D MAN/CHARLES CHAN,ANGEL/WARREN KENNETH,1
4,3-D MAN/CHARLES CHAN,ANT-MAN II/SCOTT HAR,1
...,...,...,...
224094,ZZZAX,"RODRIGUEZ, DEBRA",1
224095,ZZZAX,"ROSS, GEN. THADDEUS",2
224096,ZZZAX,"SUMMERS, NATHAN CHRI",1
224097,ZZZAX,TIGRA/GREER NELSON,1


we have the new df 

In [19]:
hero_net_g['size'].describe()

count    224099.000000
mean          2.553492
std           7.285278
min           1.000000
25%           1.000000
50%           1.000000
75%           2.000000
max        1267.000000
Name: size, dtype: float64

In [20]:
hero_net_g['size'].sort_values()[-100:]

186191     133
148855     133
218821     134
78102      135
91072      135
          ... 
196190     365
85416      366
195966     382
124735     627
142471    1267
Name: size, Length: 100, dtype: int64

In [21]:
G_1 = nx.from_pandas_edgelist(hero_net_g, 'hero1', 'hero2' , edge_attr='size') # dobbiamo scegliere la metrica giusta

In [22]:
#nx.draw(G_1) # mi da errore, forse sono troppi nodi?
#plt.show()

#### Second

In [23]:
nodes

,node,type
0,2001 10,comic
1,2001 8,comic
2,2001 9,comic
3,24-HOUR MAN/EMMANUEL,hero
4,3-D MAN/CHARLES CHAN,hero
...,...,...
19085,"ZOTA, CARLO",hero
19086,ZOTA,hero
19087,ZURAS,hero
19088,ZURI,hero


In [24]:
edges

,hero,comic
0,24-HOUR MAN/EMMANUEL,AA2 35
1,3-D MAN/CHARLES CHAN,AVF 4
2,3-D MAN/CHARLES CHAN,AVF 5
3,3-D MAN/CHARLES CHAN,COC 1
4,3-D MAN/CHARLES CHAN,H2 251
...,...,...
96099,ZZZAX,H2 326
96100,ZZZAX,H2 327
96101,ZZZAX,M/CP 8/4
96102,ZZZAX,PM 47


First, we create the normal graph from edges.csv

In [25]:
G_2 = nx.from_pandas_edgelist(edges, 'hero', 'comic')

Then we create two different list that contains only the name of heroes or comics

In [26]:
n_h=nodes[nodes['type'] == 'hero']['node'].tolist()

In [27]:
n_c=nodes[nodes['type'] == 'comic']['node'].tolist()

Now we merge the two lists in a dictionary 

In [28]:
dict_heroes_comics = {key: 'hero' if key in n_h else 'comic' for key in n_c + n_h}

Finally we add at each node of the graph its type

In [29]:
nx.set_node_attributes(G_2, dict_heroes_comics, 'type')

In [30]:
#nx.draw(G_2) # mi da errore, forse sono troppi nodi?
#plt.show()

# 2. Backend Implementation

## Functionality 1 - extract the graph's features

In [31]:
all_heroes=len(edges['hero'].value_counts())
all_heroes

6439

we save that value because it will be used in the functions to create the subgraph

In [33]:
def create_subN1(G,N):
  if N == all_heroes: # if N is not set by the user, the function should consider the whole data
    sub_g=G
  else: # otherwise we select only the top n nodes 
    series=edges['hero'].value_counts()[0:N]
    names_list=series.index.tolist()
    sub_g=G.subgraph(names_list)
  return sub_g

The difference between subN1 and subN2 is that in the latter one we consider also all the comic nodes, otherwise the graph will be empty (the edges are only  between a hero and a comic)

In [72]:
def create_subN2(G,N):
  if N == all_heroes: # if N is not set by the user, the function should consider the whole data
    sub_g=G
  else: # otherwise we select only the top n nodes of heroes + all comics
    series=edges['hero'].value_counts()[0:N]
    names_list=series.index.tolist()
    comics_list=edges['comic'].value_counts().index.tolist()
    all=names_list+comics_list
    sub_g=G.subgraph(all)
  return sub_g

In [ ]:
edges

,hero,comic
0,24-HOUR MAN/EMMANUEL,AA2 35
1,3-D MAN/CHARLES CHAN,AVF 4
2,3-D MAN/CHARLES CHAN,AVF 5
3,3-D MAN/CHARLES CHAN,COC 1
4,3-D MAN/CHARLES CHAN,H2 251
...,...,...
96099,ZZZAX,H2 326
96100,ZZZAX,H2 327
96101,ZZZAX,M/CP 8/4
96102,ZZZAX,PM 47


In [128]:
def functional1_type1(sub_g, G_type):

   output={} # initialize the dict of outputs

   # The number of nodes in the network
   num_nodes = sub_g.number_of_nodes()
   output['Number of nodes']=num_nodes

   # The number of collaborations of each superhero with the others (i.e the degrees)
   degrees=dict(sub_g.degree())
   output['Collaborations']=list(degrees.items())
  
   # The network's density
   density = nx.density(sub_g)
   output['Density']=density

   # The average degree of the network
   avg_degree=sum(degrees.values())/len(degrees.values())
   output['Average Degree']=avg_degree

   # The network's degree distribution
   degrees_list=list(degrees.values())
   degree_distribution = Counter(degrees_list) # Given the degrees list we obatin the frequency distribution
   sorted_degree_distribution = sorted(degree_distribution.items(), key=lambda x: x[0]) # we sort the frequency by the keys
   output['Degree Distribution']=sorted_degree_distribution





   # The network's Hubs
   sorted_dict = sorted(degrees.items(), key=lambda x: x[1])
   pos=int(len(sorted_dict)*0.95)
   output['Network Hubs']=sorted_dict[pos-1::]

   # Whether the Network is sparse or dense
   if density>0.5:
    output['Dense']=1.0 # 1 if it is dense
   else:
    output['Dense']=0.0 # 0 if it is sparse

   return output 


In [109]:
G_2.number_of_nodes()

19087

In [129]:
def functional1_type2(sub_g, G_type):

   output={} # initialize the dict of outputs

# The following 3 rows of code will be usefull for the functionals regarding only the comic nodes

   # we obtain for each node the type
   dictionary_types=dict(nx.get_node_attributes(sub_g, "type"))
   # Then we obtain the list with only the comic nodes
   keys_comic = {k: v for k, v in dictionary_types.items() if v == 'comic'}
   comics=list(keys_comic.keys())

# Now we can start:

# regarding the Density of the graph we will use both types of nodes

   # The network's density
   density = nx.density(sub_g)
   output['Density']=density

   # Whether the Network is sparse or dense
   if density>0.5:
    output['Dense']=1.0 # 1 if it is dense
   else:
    output['Dense']=0.0 # 0 if it is sparse

# regarding the other operations we will consider only the comic type:

   # The number of nodes for both types
   total_number= sub_g.number_of_nodes()

   degrees=dict(sub_g.degree())
   comics_subset = {k: v for k, v in degrees.items() if k in comics}
   heroes_subset = {k: v for k, v in degrees.items() if k not in comics} 

   output['Total number of nodes']=total_number

   output['Number of heroes']=len(list(heroes_subset.items()))

   output['Number of comics']=len(list(comics_subset.items()))

   # The number of heroes that have appeared in each comic (i.e the degrees)
   degrees_subset = {k: v for k, v in degrees.items() if k in comics}
   output['Number of heroes in the comic']=list(degrees_subset.items())

   # The network's degree distribution
   degrees_list=list(degrees_subset.values())
   degree_distribution = Counter(degrees_list) # Given the degrees list we obatin the frequency distribution
   sorted_degree_distribution = sorted(degree_distribution.items(), key=lambda x: x[0]) # we sort the frequency by the keys
   output['Degree Distribution']=sorted_degree_distribution # The output means how many times each degree appears

   # The average degree of the network
   avg_degree=sum(degrees_subset.values())/len(degrees_subset.values())
   output['Average Degree']=avg_degree

   # The network's Hubs
   sorted_dict = sorted(degrees_subset.items(), key=lambda x: x[1])
   pos=int(len(sorted_dict)*0.95)
   output['Network Hubs']=sorted_dict[pos-1::]

   return output


In [130]:
def ex_features(G, G_type, N = all_heroes):
  
#################### Neither 1 or 2 ########################################
  
  if G_type not in [1, 2]:
    raise ValueError('Invalid graph type: remember, this function only accepts graph types equal to 1 or 2')

#################### TYPE = 1 ##############################################

  elif G_type == 1:
   # Create the Sub 
   sub_g= create_subN1(G,N)
   
   # Apply all the functions for Type = 1
   output= functional1_type1(sub_g, G_type)

   
#################### TYPE = 2 ##############################################

  else: 
   sub_g= create_subN2(G,N)
  
   # Apply all the functions for Type = 2
   output= functional1_type2(sub_g, G_type)


  return output

In [134]:
ex_features(G_2, 2, 10)

{'Density': 0.0001270710655914538,
 'Dense': 0.0,
 'Total number of nodes': 12661,
 'Number of heroes': 10,
 'Number of comics': 12651,
 'Number of heroes in the comic': [('AA2 35', 0),
  ('AVF 4', 1),
  ('AVF 5', 1),
  ('COC 1', 10),
  ('H2 251', 1),
  ('H2 252', 1),
  ('M/PRM 35', 0),
  ('M/PRM 36', 0),
  ('M/PRM 37', 0),
  ('WI? 9', 0),
  ('CA3 36', 1),
  ('CM 51', 2),
  ('Q 14', 0),
  ('Q 16', 0),
  ('T 208', 1),
  ('T 214', 1),
  ('T 215', 1),
  ('T 216', 1),
  ('T 440', 0),
  ('SLEEP 1', 0),
  ('SLEEP 19', 0),
  ('SLEEP 2', 0),
  ('DD/SM 1', 1),
  ('W2 52', 1),
  ('W2 53', 3),
  ('XFOR 108', 0),
  ('XFOR 109', 0),
  ('ABOM 2', 0),
  ('ABOM 3', 0),
  ('ASM 23', 1),
  ('H 15', 1),
  ('H 20', 0),
  ('H2 136', 1),
  ('H2 137', 1),
  ('H2 159', 1),
  ('H2 171', 1),
  ('H2 194', 1),
  ('H2 195', 1),
  ('H2 196', 1),
  ('H2 270', 1),
  ('H2 278', 10),
  ('H2 287', 1),
  ('H2 288', 1),
  ('H2 289', 1),
  ('H2 290', 1),
  ('H2 364', 1),
  ('H2 366', 1),
  ('H2 382', 1),
  ('H2 383', 2),
 

In [135]:
ex_features(G_1, 1, 10)

{'Number of nodes': 10,
 'Collaborations': [('INVISIBLE WOMAN/SUE', 9),
  ('THOR/DR. DONALD BLAK', 9),
  ('THING/BENJAMIN J. GR', 9),
  ('HUMAN TORCH/JOHNNY S', 9),
  ('SPIDER-MAN/PETER PARKER', 9),
  ('IRON MAN/TONY STARK', 9),
  ('HULK/DR. ROBERT BRUC', 9),
  ('MR. FANTASTIC/REED R', 9),
  ('WOLVERINE/LOGAN', 9),
  ('CAPTAIN AMERICA', 9)],
 'Density': 1.0,
 'Average Degree': 9.0,
 'Degree Distribution': [(9, 10)],
 'Network Hubs': [('WOLVERINE/LOGAN', 9), ('CAPTAIN AMERICA', 9)],
 'Dense': 1.0}